Convert JSON to masked dataset

In [1]:
import os
from glob import glob

# find all JSON annotation files
json_files = glob("*.json")

for jf in json_files:
    # output folder: e.g., 1.json -> 1_dataset
    out_dir = jf.replace(".json", "_dataset")
    cmd = f"labelme_json_to_dataset \"{jf}\" -o \"{out_dir}\""
    print(f"Converting: {jf} -> {out_dir}")
    os.system(cmd)


move the files to the images and masks folder

In [2]:
import os
import shutil
from glob import glob

# make output folders
os.makedirs("images", exist_ok=True)
os.makedirs("masks", exist_ok=True)

# find all _dataset folders
dataset_folders = glob("*_dataset")

for folder in dataset_folders:
    img_path = os.path.join(folder, "img.png")
    mask_path = os.path.join(folder, "label.png")

    if os.path.exists(img_path) and os.path.exists(mask_path):
        # name them based on folder name, e.g., 1_dataset -> 1.png
        base_name = folder.replace("_dataset", "")

        shutil.copy(img_path, f"images/{base_name}.png")
        shutil.copy(mask_path, f"masks/{base_name}.png")

print("✅ All images and masks collected into 'images/' and 'masks/'")


✅ All images and masks collected into 'images/' and 'masks/'


Split the dataset into 80 20

In [3]:
import os
import shutil
import random
from glob import glob

# paths (update for your actual folders)
images_dir = "images"
masks_dir = "masks"

# output folders
for s in ["train", "val"]:
    os.makedirs(f"images/{s}", exist_ok=True)
    os.makedirs(f"masks/{s}", exist_ok=True)

# find all images
images = glob(os.path.join(images_dir, "*.png"))
images.sort()  # consistent ordering

# debug: check what files were found
print("Found images:", images)

# shuffle with seed = 1
random.seed(1)
random.shuffle(images)

# split 80/20
split_idx = int(len(images) * 0.8)
train_files = images[:split_idx]
val_files = images[split_idx:]

def copy_files(file_list, split):
    for img_path in file_list:
        fname = os.path.basename(img_path)
        mask_path = os.path.join(masks_dir, fname)

        if not os.path.exists(mask_path):
            print(f"⚠️  Mask not found for image: {fname}")
            continue

        # copy image + mask
        shutil.copy(img_path, f"images/{split}/{fname}")
        shutil.copy(mask_path, f"masks/{split}/{fname}")

copy_files(train_files, "train")
copy_files(val_files, "val")

print("✅ Dataset split completed (copied, originals intact):")
print(f"Train: {len(train_files)} images")
print(f"Val:   {len(val_files)} images")


Found images: []
✅ Dataset split completed (copied, originals intact):
Train: 0 images
Val:   0 images


In [9]:
import os
import re

def rename_variant_files(img_dir, mask_dir):
    # Pattern to match base filename (e.g., 188.png from 188(1).png)
    def base_name(fname):
        return re.sub(r'\(\d+\)', '', fname)

    # Rename image files
    for fname in os.listdir(img_dir):
        new_name = base_name(fname)
        if fname != new_name:
            src = os.path.join(img_dir, fname)
            dst = os.path.join(img_dir, new_name)
            if not os.path.exists(dst):
                os.rename(src, dst)
            else:
                os.remove(src)  # Remove duplicate

    # Rename mask files
    for fname in os.listdir(mask_dir):
        new_name = base_name(fname)
        if fname != new_name:
            src = os.path.join(mask_dir, fname)
            dst = os.path.join(mask_dir, new_name)
            if not os.path.exists(dst):
                os.rename(src, dst)
            else:
                os.remove(src)  # Remove duplicate

# Apply to train and val folders
rename_variant_files("images/train", "masks/train")
rename_variant_files("images/val", "masks/val")

U-Net

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

# -----------------------
# Dataset
# -----------------------
class SegmentationDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.images = os.listdir(img_dir)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])  # assumes same filename

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # grayscale

        if self.transform is not None:
            image = self.transform(image)
            mask = self.transform(mask)

        mask = (mask > 0).float()  # binarize
        return image, mask

# -----------------------
# U-Net
# -----------------------
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.net(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()
        self.enc1 = DoubleConv(in_channels, 64)
        self.enc2 = DoubleConv(64, 128)
        self.enc3 = DoubleConv(128, 256)
        self.enc4 = DoubleConv(256, 512)

        self.pool = nn.MaxPool2d(2)
        self.bottleneck = DoubleConv(512, 1024)

        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = DoubleConv(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = DoubleConv(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = DoubleConv(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = DoubleConv(128, 64)

        self.conv_final = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        e4 = self.enc4(self.pool(e3))

        b = self.bottleneck(self.pool(e4))

        d4 = self.upconv4(b)
        d4 = torch.cat((d4, e4), dim=1)
        d4 = self.dec4(d4)

        d3 = self.upconv3(d4)
        d3 = torch.cat((d3, e3), dim=1)
        d3 = self.dec3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.dec2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.dec1(d1)

        return torch.sigmoid(self.conv_final(d1))

# -----------------------
# Metrics
# -----------------------
def dice_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = target.float()
    intersection = (pred * target).sum()
    return ((2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)).item()

def iou_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = target.float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return ((intersection + smooth) / (union + smooth)).item()

def pixel_accuracy(pred, target):
    pred = (pred > 0.5).float()
    correct = (pred == target).float().sum()
    return (correct / target.numel()).item()

def precision_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = target.float()
    tp = (pred * target).sum()
    fp = (pred * (1 - target)).sum()
    return (tp / (tp + fp + smooth)).item()

def recall_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = target.float()
    tp = (pred * target).sum()
    fn = ((1 - pred) * target).sum()
    return (tp / (tp + fn + smooth)).item()

# -----------------------
# Training Setup
# -----------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

train_dataset = SegmentationDataset("D:/Wave_CMKL/Year2_term1/Computer_Vision/images/train", "D:/Wave_CMKL/Year2_term1/Computer_Vision/masks/train", transform=transform)
val_dataset = SegmentationDataset("D:/Wave_CMKL/Year2_term1/Computer_Vision/images/val", "D:/Wave_CMKL/Year2_term1/Computer_Vision/masks/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

model = UNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
bce = nn.BCELoss()

# -----------------------
# Training Loop
# -----------------------
num_epochs = 20
best_val_loss = float("inf")

for epoch in range(num_epochs):
    model.train()
    train_loss, train_dice, train_iou, train_acc = 0, 0, 0, 0
    for imgs, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        imgs, masks = imgs.to(device), masks.to(device)

        preds = model(imgs)
        loss = 0.5 * bce(preds, masks) + 0.5 * (1 - dice_score(preds, masks))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_dice += dice_score(preds, masks)
        train_iou += iou_score(preds, masks)
        train_acc += pixel_accuracy(preds, masks)

    # Validation
    model.eval()
    val_loss, val_dice, val_iou, val_acc, val_prec, val_rec = 0, 0, 0, 0, 0, 0
    with torch.no_grad():
        for imgs, masks in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            imgs, masks = imgs.to(device), masks.to(device)
            preds = model(imgs)

            loss = 0.5 * bce(preds, masks) + 0.5 * (1 - dice_score(preds, masks))
            val_loss += loss.item()

            val_dice += dice_score(preds, masks)
            val_iou += iou_score(preds, masks)
            val_acc += pixel_accuracy(preds, masks)
            val_prec += precision_score(preds, masks)
            val_rec += recall_score(preds, masks)

    print(f"\nEpoch {epoch+1}:")
    print(f" Train Loss={train_loss/len(train_loader):.4f} | "
          f"Dice={train_dice/len(train_loader):.4f} | IoU={train_iou/len(train_loader):.4f} | Acc={train_acc/len(train_loader):.4f}")
    print(f" Val   Loss={val_loss/len(val_loader):.4f} | "
          f"Dice={val_dice/len(val_loader):.4f} | IoU={val_iou/len(val_loader):.4f} | Acc={val_acc/len(val_loader):.4f} | "
          f"Prec={val_prec/len(val_loader):.4f} | Recall={val_rec/len(val_loader):.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_unet.pth")
        print("✅ Model saved!")



c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\cuda\__init__.py:235: UserWarning: 
NVIDIA GeForce RTX 5070 Ti with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90 compute_37.
If you want to use the NVIDIA GeForce RTX 5070 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(
Epoch 1/20 - Validation: 100%|██████████| 8/8 [00:01<00:00,  6.68it/s]



Epoch 1:
 Train Loss=0.6926 | Dice=0.0769 | IoU=0.0409 | Acc=0.9544
 Val   Loss=0.7288 | Dice=0.0000 | IoU=0.0000 | Acc=0.9948 | Prec=0.0000 | Recall=0.0000
✅ Model saved!


Epoch 2/20 - Validation: 100%|██████████| 8/8 [00:01<00:00,  6.81it/s]



Epoch 2:
 Train Loss=0.4391 | Dice=0.4480 | IoU=0.2980 | Acc=0.9952
 Val   Loss=0.4129 | Dice=0.4851 | IoU=0.3360 | Acc=0.9959 | Prec=0.5562 | Recall=0.4524
✅ Model saved!


Epoch 3/20 - Validation: 100%|██████████| 8/8 [00:01<00:00,  6.65it/s]


...


Epoch 18:
 Train Loss=0.1006 | Dice=0.9116 | IoU=0.8378 | Acc=0.9992
 Val   Loss=0.1449 | Dice=0.8223 | IoU=0.7023 | Acc=0.9983 | Prec=0.8657 | Recall=0.7903
✅ Model saved!


Epoch 19/20 - Validation: 100%|██████████| 8/8 [00:01<00:00,  5.96it/s]



Epoch 19:
 Train Loss=0.0968 | Dice=0.9135 | IoU=0.8411 | Acc=0.9992
 Val   Loss=0.1403 | Dice=0.8268 | IoU=0.7089 | Acc=0.9983 | Prec=0.8360 | Recall=0.8302
✅ Model saved!


Epoch 20/20 - Validation: 100%|██████████| 8/8 [00:01<00:00,  5.96it/s]


Epoch 20:
 Train Loss=0.0920 | Dice=0.9176 | IoU=0.8480 | Acc=0.9993
 Val   Loss=0.1438 | Dice=0.8147 | IoU=0.6904 | Acc=0.9983 | Prec=0.8742 | Recall=0.7735


In [3]:
from torchvision.models.segmentation import deeplabv3_resnet50

# -----------------------
# Dataset
# -----------------------
class SegmentationDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.images = os.listdir(img_dir)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])  # assumes same filename

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # grayscale

        if self.transform is not None:
            image = self.transform(image)
            mask = self.transform(mask)

        mask = (mask > 0).float()  # binarize
        return image, mask

# -----------------------
# Metrics
# -----------------------
def dice_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = target.float()
    intersection = (pred * target).sum()
    return ((2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)).item()

def iou_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = target.float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return ((intersection + smooth) / (union + smooth)).item()

def pixel_accuracy(pred, target):
    pred = (pred > 0.5).float()
    correct = (pred == target).float().sum()
    return (correct / target.numel()).item()

def precision_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = target.float()
    tp = (pred * target).sum()
    fp = (pred * (1 - target)).sum()
    return (tp / (tp + fp + smooth)).item()

def recall_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = target.float()
    tp = (pred * target).sum()
    fn = ((1 - pred) * target).sum()
    return (tp / (tp + fn + smooth)).item()

# -----------------------
# Training Setup
# -----------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

train_dataset = SegmentationDataset("D:/Wave_CMKL/Year2_term1/Computer_Vision/images/train", "D:/Wave_CMKL/Year2_term1/Computer_Vision/masks/train", transform=transform)
val_dataset = SegmentationDataset("D:/Wave_CMKL/Year2_term1/Computer_Vision/images/val", "D:/Wave_CMKL/Year2_term1/Computer_Vision/masks/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

# -----------------------
# DeepLabV3 Model
# -----------------------
model = deeplabv3_resnet50(weights=None, num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
bce = nn.BCEWithLogitsLoss()  # Use logits for DeepLabV3

# -----------------------
# Training Loop
# -----------------------
num_epochs = 20
best_val_loss = float("inf")

for epoch in range(num_epochs):
    model.train()
    train_loss, train_dice, train_iou, train_acc = 0, 0, 0, 0
    for imgs, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        imgs, masks = imgs.to(device), masks.to(device)

        outputs = model(imgs)["out"]  # shape: [B, 1, H, W]
        preds = torch.sigmoid(outputs)
        loss = 0.5 * bce(outputs, masks) + 0.5 * (1 - dice_score(preds, masks))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_dice += dice_score(preds, masks)
        train_iou += iou_score(preds, masks)
        train_acc += pixel_accuracy(preds, masks)

    # Validation
    model.eval()
    val_loss, val_dice, val_iou, val_acc, val_prec, val_rec = 0, 0, 0, 0, 0, 0
    with torch.no_grad():
        for imgs, masks in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)["out"]
            preds = torch.sigmoid(outputs)

            loss = 0.5 * bce(outputs, masks) + 0.5 * (1 - dice_score(preds, masks))
            val_loss += loss.item()

            val_dice += dice_score(preds, masks)
            val_iou += iou_score(preds, masks)
            val_acc += pixel_accuracy(preds, masks)
            val_prec += precision_score(preds, masks)
            val_rec += recall_score(preds, masks)

    print(f"\nEpoch {epoch+1}:")
    print(f" Train Loss={train_loss/len(train_loader):.4f} | "
          f"Dice={train_dice/len(train_loader):.4f} | IoU={train_iou/len(train_loader):.4f} | Acc={train_acc/len(train_loader):.4f}")
    print(f" Val   Loss={val_loss/len(val_loader):.4f} | "
          f"Dice={val_dice/len(val_loader):.4f} | IoU={val_iou/len(val_loader):.4f} | Acc={val_acc/len(val_loader):.4f} | "
          f"Prec={val_prec/len(val_loader):.4f} | Recall={val_rec/len(val_loader):.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_deeplabv3.pth")
        print("✅ Model saved!")

Epoch 1/20 - Validation: 100%|██████████| 8/8 [00:01<00:00,  5.07it/s]



Epoch 1:
 Train Loss=0.6889 | Dice=0.0146 | IoU=0.0075 | Acc=0.9664
 Val   Loss=0.5896 | Dice=0.0000 | IoU=0.0000 | Acc=0.9909 | Prec=0.0000 | Recall=0.0000
✅ Model saved!


Epoch 2/20 - Validation: 100%|██████████| 8/8 [00:01<00:00,  5.97it/s]



Epoch 2:
 Train Loss=0.5841 | Dice=0.0002 | IoU=0.0001 | Acc=0.9942
 Val   Loss=0.5557 | Dice=0.0000 | IoU=0.0000 | Acc=0.9942 | Prec=0.0000 | Recall=0.0000
✅ Model saved!


Epoch 3/20 - Validation: 100%|██████████| 8/8 [00:01<00:00,  6.11it/s]


...


Epoch 18:
 Train Loss=0.0802 | Dice=0.8615 | IoU=0.7571 | Acc=0.9988
 Val   Loss=0.1410 | Dice=0.7432 | IoU=0.5986 | Acc=0.9974 | Prec=0.7380 | Recall=0.7850


Epoch 19/20 - Validation: 100%|██████████| 8/8 [00:01<00:00,  6.33it/s]



Epoch 19:
 Train Loss=0.0806 | Dice=0.8592 | IoU=0.7542 | Acc=0.9988
 Val   Loss=0.1477 | Dice=0.7291 | IoU=0.5823 | Acc=0.9972 | Prec=0.7442 | Recall=0.7682


Epoch 20/20 - Validation: 100%|██████████| 8/8 [00:01<00:00,  6.64it/s]



Epoch 20:
 Train Loss=0.0785 | Dice=0.8623 | IoU=0.7585 | Acc=0.9988
 Val   Loss=0.1325 | Dice=0.7560 | IoU=0.6124 | Acc=0.9976 | Prec=0.7516 | Recall=0.7878
✅ Model saved!


In [5]:
import segmentation_models_pytorch as smp

# -----------------------
# FPN Model
# -----------------------
model = smp.FPN(
    encoder_name="resnet34",        # backbone
    encoder_weights="imagenet",     # use pretrained weights
    in_channels=3,                  # RGB images
    classes=1,                      # binary mask
    activation=None                 # use logits for BCEWithLogitsLoss
).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
bce = nn.BCEWithLogitsLoss()

# -----------------------
# Training Loop
# -----------------------
num_epochs = 20
best_val_loss = float("inf")

for epoch in range(num_epochs):
    model.train()
    train_loss, train_dice, train_iou, train_acc = 0, 0, 0, 0
    for imgs, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        imgs, masks = imgs.to(device), masks.to(device)

        outputs = model(imgs)  # shape: [B, 1, H, W]
        preds = torch.sigmoid(outputs)
        loss = 0.5 * bce(outputs, masks) + 0.5 * (1 - dice_score(preds, masks))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_dice += dice_score(preds, masks)
        train_iou += iou_score(preds, masks)
        train_acc += pixel_accuracy(preds, masks)

    # Validation
    model.eval()
    val_loss, val_dice, val_iou, val_acc, val_prec, val_rec = 0, 0, 0, 0, 0, 0
    with torch.no_grad():
        for imgs, masks in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)
            preds = torch.sigmoid(outputs)

            loss = 0.5 * bce(outputs, masks) + 0.5 * (1 - dice_score(preds, masks))
            val_loss += loss.item()

            val_dice += dice_score(preds, masks)
            val_iou += iou_score(preds, masks)
            val_acc += pixel_accuracy(preds, masks)
            val_prec += precision_score(preds, masks)
            val_rec += recall_score(preds, masks)

    print(f"\nEpoch {epoch+1}:")
    print(f" Train Loss={train_loss/len(train_loader):.4f} | "
          f"Dice={train_dice/len(train_loader):.4f} | IoU={train_iou/len(train_loader):.4f} | Acc={train_acc/len(train_loader):.4f}")
    print(f" Val   Loss={val_loss/len(val_loader):.4f} | "
          f"Dice={val_dice/len(val_loader):.4f} | IoU={val_iou/len(val_loader):.4f} | Acc={val_acc/len(val_loader):.4f} | "
          f"Prec={val_prec/len(val_loader):.4f} | Recall={val_rec/len(val_loader):.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_fpn.pth")
        print("✅ Model saved!")

config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--smp-hub--resnet34.imagenet. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

Epoch 1/20 - Validation: 100%|██████████| 8/8 [00:04<00:00,  1.84it/s]



Epoch 1:
 Train Loss=0.5100 | Dice=0.0227 | IoU=0.0123 | Acc=0.9839
 Val   Loss=0.5112 | Dice=0.0000 | IoU=0.0000 | Acc=0.9951 | Prec=0.0000 | Recall=0.0000
✅ Model saved!


...


Epoch 18:
 Train Loss=0.0713 | Dice=0.8603 | IoU=0.7556 | Acc=0.9988
 Val   Loss=0.1156 | Dice=0.7743 | IoU=0.6383 | Acc=0.9981 | Prec=0.8900 | Recall=0.6978


Epoch 19/20 - Validation: 100%|██████████| 8/8 [00:12<00:00,  1.59s/it]



Epoch 19:
 Train Loss=0.0681 | Dice=0.8665 | IoU=0.7651 | Acc=0.9988
 Val   Loss=0.1119 | Dice=0.7820 | IoU=0.6455 | Acc=0.9981 | Prec=0.8908 | Recall=0.7019


Epoch 20/20 - Validation: 100%|██████████| 8/8 [00:01<00:00,  7.55it/s]


Epoch 20:
 Train Loss=0.0660 | Dice=0.8706 | IoU=0.7714 | Acc=0.9989
 Val   Loss=0.1141 | Dice=0.7775 | IoU=0.6415 | Acc=0.9981 | Prec=0.8832 | Recall=0.7065


In [6]:
# Compare Dice scores for U-Net, DeepLabV3, and FPN on validation set

import torch

# Load models (make sure you have trained and saved them)
unet = UNet().to(device)
unet.load_state_dict(torch.load("best_unet.pth", map_location=device))
unet.eval()

deeplab = deeplabv3_resnet50(weights=None, num_classes=1).to(device)
deeplab.load_state_dict(torch.load("best_deeplabv3.pth", map_location=device))
deeplab.eval()

import segmentation_models_pytorch as smp
fpn = smp.FPN(
    encoder_name="resnet34",
    encoder_weights=None,
    in_channels=3,
    classes=1,
    activation=None
).to(device)
fpn.load_state_dict(torch.load("best_fpn.pth", map_location=device))
fpn.eval()

dice_scores = {"UNet": 0, "DeepLabV3": 0, "FPN": 0}
num_batches = len(val_loader)

with torch.no_grad():
    for imgs, masks in val_loader:
        imgs, masks = imgs.to(device), masks.to(device)

        # U-Net
        preds_unet = unet(imgs)
        dice_scores["UNet"] += dice_score(preds_unet, masks)

        # DeepLabV3
        preds_deeplab = torch.sigmoid(deeplab(imgs)["out"])
        dice_scores["DeepLabV3"] += dice_score(preds_deeplab, masks)

        # FPN
        preds_fpn = torch.sigmoid(fpn(imgs))
        dice_scores["FPN"] += dice_score(preds_fpn, masks)

for k in dice_scores:
    dice_scores[k] /= num_batches

print("Validation Dice Scores:")
for k, v in dice_scores.items():
    print(f"{k}: {v:.4f}")

Validation Dice Scores:
UNet: 0.8268
DeepLabV3: 0.7560
FPN: 0.7869
